**Note**  
Most the code is create through exploring the documentation and Stack Overflow
- I use my own pictures for manipulation 
- This might will not run without the files located on my [github](https://github.com/lokinario/pillow_presentation)

## Installing Pillow

* you might need some more dependencies all covered in the [docs](https://pillow.readthedocs.io/en/stable/installation.html)

In [ ]:
!pip3 install Pillow

## import

In [ ]:
from PIL import Image, ImageFilter
from IPython.display import clear_output
import os

## Using the Image Class

In [ ]:
try:
    img = Image.open("./pics/disneyrob.jpg")
    print(type(img))
except:
    print("yikes didn't open boss")

In [ ]:
img

## Getting Picture Information
Let's get the information about the picture

In [ ]:
print("The size of the Image is: ")
print(img.format, img.size, img.mode)


## Saving an image

Image.save(fp, format=None, \**params)

- Saves an image under the given filename. If no format is specified, the format to use is determined from the filename extension, if possible.

In [ ]:
img.save("./pngs/newyear.png")

In [ ]:
s = Image.open('./pngs/newyear.png')
s.format

# **Modes** 
The mode of an image defines the type and depth of a pixel in the image 
- 1: 1-bit pixels, black and white
- L: 8-bit pixels, black and white
- P: 8-bit pixels, mapped to any other mode using a color palette 
- RGB: 3x8-bit pixels: true color
- RGBA: 4x8-bit pixels: true color w transparency mask 
- CMYK: 4x8-bit pixels: color separation 
- YCbCr 3x8-bit pixels, color video format 
    - Refers to JPEG 


Pillow allows you to convert images between different pixel representations using the convert() method.  
The library supports transformations between each supported mode and the “L” and “RGB” modes.  
To convert between other modes, you may have to use an intermediate image (typically an “RGB” image).



In [ ]:
test = img.convert("L")
test

## **Bands** 

Images can consist of one or more bands of data. Pillow allows you to store several bands in a single image, provided they all have the same dimensions and depth. For example, a PNG image could have ‘R’, ‘G’, ‘B’, and ‘A’ bands for the red, green, blue, and alpha transparency values. Many operations act on each band separately, e.g., histograms. It is often useful to think of each pixel as having one value per band


**Let stript the bands of the RGB image and merge() them in a different order**
- split() : splits an image's bands into a tuple
- merge(mode,bands):
    - mode: mode to use for output image
    - bands: sequence of bands all being the same size

In [ ]:
# papa smurf
r, g, b = img.split()
im = Image.merge("RGB", (b, g, r))
im

## What's that you have a logo and tired of manually putting them on??

Let's do it with code


In [ ]:
logo = Image.open('logo.png')
logo

In [ ]:
# function to get a position to place to logo
def getPosition(img,logo):
    return ((img.width - logo.width), (img.height - logo.height))

In [ ]:
img = Image.open('./pics/disneyrob.jpg')
# we make a copy
# We use this method since we're pasting a logo into our image, 
# but still retain the original.
img_copy = img.copy()
position = getPosition(img,logo)
# ok lets place the logo in the position I don't see what could go wrong
img_copy.paste(logo, position)
#let's see the outcome
img_copy

### Wait a minute the logo has a transparent background what happened??

By default, when you perform a paste, transparent pixels are pasted as solid pixels, thus the black box surrounding the logo.  
Soooo we need an additional parameter in paste for masking

In [ ]:
img = Image.open('./pics/disneyrob.jpg')
good_copy = img.copy()
position = getPosition(img,logo)
# add a third argument for masking
good_copy.paste(logo, position,logo)
# now let's see it
good_copy

### Let's paste logos to all of our pictures

In [ ]:
# function to resize to logo since our pictures are of different sizes
def getLogo(p,l):
    wh = int(p.width * 0.3)
    return l.resize((wh,wh))

In [ ]:
for f in os.listdir('./pics'):
    #sanity check that it is a jpg
    if f.endswith('jpg'):
        
        #open the image
        pic = Image.open('./pics/' + f)
        
        #save the name and the extention for saving
        fn, fext = os.path.splitext(f)
        
        newLogo = getLogo(pic,logo)
        
        #get position to place logo
        position = getPosition(pic,newLogo)
        
        #paste logo 
        pic.paste(newLogo, position,newLogo)
        
        # save into our logoed folder
        pic.save('./logoed/{}_logo{}'.format(fn, fext))

In [ ]:
demo = Image.open('./logoed/seaside_me_logo.jpg')
demo

## Pictures are too big lets make them smaller

To do this we'll try the resize method

In [ ]:
image = Image.open('./pics/handsome.jpg')

In [ ]:
new_image = image.resize((400, 400))
print(image.size) 
print(new_image.size) 
new_image

## whoa that didn't work 
- Lets try the thumbnail() method
- The following code with resize the image and maintain the aspect ratio with 400 x 400 max resolution 

In [ ]:
image.thumbnail((400, 400))
image

## So much better but lets do this for all the images

In [ ]:
#Set our bounds for resizing the image
size = (300,300)

for f in os.listdir('./logoed'):
    #sanity check that it is a jpg
    if f.endswith('jpg'):
        #open the image
        pic = Image.open('./logoed/' + f)
        #save the name and the extention for saving
        fn, fext = os.path.splitext(f)
        # Calculate the output pixel value using a 
        # high-quality Lanczos filter (a truncated sinc) 
        # on all pixels that may contribute to the output value. 
        pic.thumbnail(size,Image.LANCZOS)
        # lastly save it into the thumbnail folder
        pic.save('thumb/{}_tmb{}'.format(fn, fext))

In [ ]:
pic

In [ ]:
os.listdir('./thumb/')

## ImageFilter Module  

The ImageFilter module contains definitions for a pre-defined set of filters, which can be be used with the Image.filter() method.

### Filters
.filter(ImageFilter.___)
- Predefined
    - BLUR
    - CONTOUR
    - DETAIL
    - EDGE_ENHANCE
    - EDGE_ENHANCE_MORE
    - EMBOSS
    - FIND_EDGES
    - SHARPEN
    - SMOOTH
    - SMOOTH_MORE

The use case for Pillow here would be for Deep Learning on images

In [ ]:
try:
    # Load an image with file path
    imgs = []
    pic = Image.open("./thumb/handsome_logo_tmb.jpg")
    filters = [ImageFilter.BLUR,ImageFilter.CONTOUR,ImageFilter.DETAIL,ImageFilter.EDGE_ENHANCE,
               ImageFilter.EDGE_ENHANCE_MORE,ImageFilter.EMBOSS,ImageFilter.FIND_EDGES,ImageFilter.SHARPEN,
              ImageFilter.SMOOTH,ImageFilter.SMOOTH_MORE]
    for f in filters:
        fpic = pic.filter(f)
        print(f)
        fpic.show()
        input("Press Enter for next filter")
        clear_output()
        imgs.append(fpic)

except:
    print ("Unable to load image")

In [ ]:
imgs[6]

In [ ]:
imgs[1]

## wow much color lets see an image histogram

[reference 1](https://pythontic.com/image-processing/pillow/histogram)  
[reference 2](https://stackoverflow.com/questions/22460577/understanding-histogram-in-pillow)


- the Python Image Processing Library provides several methods to extract information pertaining to an image. The histogram() method provides information on counts of different colors/bands.
 

- histogram() method returns a list of pixel counts for each band present in the image. The list will have all the counts concatenated for each band.
 

- If an image is of mode "RGB" then for each of band/color a list of pixel counts will be returned, totaling 768.
 

- In other words, for an RGB image, the histogram() method gives information on how many kind of Red, Green and Blue pixels are present in the image for each 256 types of red, 256 types of green and 256 types of blue.

In [ ]:
# Create an Image with specific RGB value
image = Image.open("./pics/disneyrob.jpg")
image

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
 
# define functions to give the corresponding color for the bars
def getRed(redVal):
    return '#%02x%02x%02x' % (redVal, 0, 0)

def getGreen(greenVal):
    return '#%02x%02x%02x' % (0, greenVal, 0)

def getBlue(blueVal):
    return '#%02x%02x%02x' % (0, 0, blueVal)

In [ ]:
# Get the color histogram of the image

histogram = image.histogram()

# Take only the Red counts
l1 = histogram[0:256] 

# Take only the Blue counts
l2 = histogram[256:512] 

# Take only the Green counts
l3 = histogram[512:768]

In [ ]:
plt.figure(0)

# RGB histogram
for i in range(0, 256):
    plt.bar(i, l1[i], color = getRed(i), edgecolor=getRed(i), alpha=0.3)
    plt.bar(i, l2[i], color = getGreen(i), edgecolor=getGreen(i),alpha=0.3)
    plt.bar(i, l3[i], color = getBlue(i), edgecolor=getBlue(i),alpha=0.3)
plt.show()

# Ok fun time lets make a gif 
- [source](http://www.pythoninformer.com/python-libraries/pillow/creating-animated-gif/)

In [ ]:
# create a list of the pics we want 
images = []

for f in os.listdir('./logoed'):
    #find pics i picked by check if it starts w IMG
    if f.startswith('IMG'):
        # open the picture
        frame = Image.open('./logoed/' + f)
        #add it to our list
        images.append(frame)

In [ ]:
# Save the frames as an animated GIF
images[0].save('./gif/nobait.gif',
               save_all=True,
               append_images=images[1:],
               duration=150,
               loop=0)

[Lets check it out](./gif/nobait.gif "segment")